In [1]:
import argparse
import warnings

import torch
import torch.nn as nn
from sklearn.metrics import precision_score, f1_score, recall_score, confusion_matrix

from dataset import get_dataloaders
from utils import get_model

warnings.filterwarnings("ignore")

parser = argparse.ArgumentParser(description='USTC Computer Vision Final Project')
parser.add_argument('--batch_size', default=128, type=int)
parser.add_argument('--seed', default=0, type=int)
parser.add_argument('--data_path', default='datasets/fer2013/fer2013.csv', type=str)
parser.add_argument('--checkpoint', default='', type=str)
parser.add_argument('--arch', default="ResNet18", type=str)
parser.add_argument('--Ncrop', default=True, type=eval)


def correct_count(output, target, topk=(1,)):
    """Computes the top k corrrect count for the specified values of k"""
    maxk = max(topk)

    _, pred = output.topk(maxk, 1, True, True)
    pred = pred.t()
    correct = pred.eq(target.view(1, -1).expand_as(pred))

    res = []
    for k in topk:
        correct_k = correct[:k].contiguous().view(-1).float().sum(0, keepdim=True)
        res.append(correct_k)
    return res


def evaluate(net, dataloader, loss_fn, Ncrop, device):
    net = net.eval()
    loss_tr, n_samples = 0.0, 0.0

    y_pred = []
    y_gt = []

    correct_count1 = 0
    correct_count2 = 0

    for data in dataloader:
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        if Ncrop:
            # fuse crops and batchsize
            bs, ncrops, c, h, w = inputs.shape
            inputs = inputs.view(-1, c, h, w)

            # forward
            outputs = net(inputs)

            # combine results across the crops
            outputs = outputs.view(bs, ncrops, -1)
            outputs = torch.sum(outputs, dim=1) / ncrops
        else:
            outputs = net(inputs)

        loss = loss_fn(outputs, labels)

        # calculate performance metrics
        loss_tr += loss.item()

        # accuracy
        counts = correct_count(outputs, labels, topk=(1, 2))
        correct_count1 += counts[0].item()
        correct_count2 += counts[1].item()

        _, preds = torch.max(outputs.data, 1)
        n_samples += labels.size(0)

        y_pred.extend(pred.item() for pred in preds)
        y_gt.extend(y.item() for y in labels)

    acc1 = 100 * correct_count1 / n_samples
    acc2 = 100 * correct_count2 / n_samples
    loss = loss_tr / n_samples
    print("--------------------------------------------------------")
    print("Top 1 Accuracy: %2.6f %%" % acc1)
    print("Top 2 Accuracy: %2.6f %%" % acc2)
    print("Loss: %2.6f" % loss)
    print("Precision: %2.6f" % precision_score(y_gt, y_pred, average='micro'))
    print("Recall: %2.6f" % recall_score(y_gt, y_pred, average='micro'))
    print("F1 Score: %2.6f" % f1_score(y_gt, y_pred, average='micro'))
    print("Confusion Matrix:\n", confusion_matrix(y_gt, y_pred), '\n')


In [2]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model = get_model().to(device)
model

ResNet(
  (conv1): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (shortcut): Sequential()
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=

In [16]:
def main():
    # args = parser.parse_args()
    loss_fn = nn.CrossEntropyLoss()

    checkpoint = torch.load('./weights/best_checkpoint.tar',map_location=torch.device('cpu'))
    model.load_state_dict(checkpoint['model_state_dict'])

    test_loader = get_dataloaders(augment=False)
    with torch.no_grad():

        print("Test")
        evaluate(model, test_loader, loss_fn, True, device)


if __name__ == "__main__":
    main()

FileNotFoundError: [Errno 2] No such file or directory: '/home/laptop-kl-11/personal_project/face_expression_classification/dataset/test'

In [8]:
def main():
    # args = parser.parse_args()
    loss_fn = nn.CrossEntropyLoss()

    checkpoint = torch.load('/home/laptop-kl-11/personal_project/face_expression_classification/ResNet18/weights/best_checkpoint.tar',map_location=torch.device('cpu'))
    model.load_state_dict(checkpoint['model_state_dict'])

    test_loader = get_dataloaders(augment=False)
    with torch.no_grad():

        print("Test")
        evaluate(model, test_loader, loss_fn, True, device)


if __name__ == "__main__":
    main()

Test
--------------------------------------------------------
Top 1 Accuracy: 73.711340 %
Top 2 Accuracy: 86.263583 %
Loss: 0.014132
Precision: 0.737113
Recall: 0.737113
F1 Score: 0.737113
Confusion Matrix:
 [[ 643    5   76   25  127   16   66]
 [  24   73    3    3    5    1    2]
 [  91    2  576   23  185   67   80]
 [  27    0   15 1584   27   38   83]
 [  99    0  102   32  805   13  196]
 [  15    0   54   32   15  701   14]
 [  50    1   41   57  159   16  909]] 

